In [1]:
import os,shutil,cv2
import numpy as np
from dpp.dataset.softdata.dbutils_qk import connect_database,query_image
from dpp.dataset.softdata.func import gen_full_img,get_no_json_long_img,gen_long_img

### 量跑数据中生成excel，json

In [2]:
with connect_database(host='192.168.2.98') as db_connect:
    # print(query_production(db_connect, 6))
        query_image(db_connect,367, save_type='all', save_image=False, save_thumb=False, save_ok_image = False,
                    retrain_filtered=True,
                    split_thresh=10000000000000000,
                    image_folders=[
                        r"\\TEST03slave\MiddleFrame-A-OQC\01-project\x64\Release\Data\X12\Image"],
                    QC_data="",
                    save_path=r"D:\PAD\soft"
                    )

批次信息-367,数据查询中...
查询完成-135条缺陷,数据解析中...
查询完成-1526条OK图,数据解析中...
解析完成,共10个产品,1567张图像!


计算特征: 100%|██████████████████████████| 1567/1567 [00:00<00:00, 22770.88it/s]

batch_0301.json is saving...


save successfully! ->PATH: D:\PAD\soft\batch_0301.json


保存Excel[batch_0301]数据: 100%|██████████| 1568/1568 [00:00<00:00, 1802.91it/s]


### 挑选量跑数据中的NG/ORIG数据

In [13]:
src = r"\\TEST03slave\MiddleFrame-A-OQC\01-project\x64\Release\Data\X12\Image\370\0301"
dst = r"D:\PAD\soft\images"
# os.makedirs(dst)
out_ng_path = os.path.join(dst,"NG")
os.makedirs(out_ng_path)
out_orig_path = os.path.join(dst,"ORIG")
os.makedirs(out_orig_path)
products = os.listdir(src)
for p in products:  # 产品号
    ng_path = os.path.join(src, p, "outer","NG")
    orig_path = os.path.join(src, p, "outer","ORIG")
    imgs = os.listdir(ng_path)
    for img in imgs:
        img_bmp = img.replace(".jpg", ".jpg")   # 原图格式默认jpg
        shutil.copy(os.path.join(ng_path, img),
                    os.path.join(out_ng_path, "{}-{}".format(str(p), img)))
        shutil.copy(os.path.join(orig_path, img_bmp),
                    os.path.join(out_orig_path, "{}-{}".format(str(p), img_bmp)))
    

### 按照NG分类结果对ORIG原图分类

In [2]:
result_path = r"D:\pad_before\train\del_rect"      # 结果图分类路径
dst = r"D:\pad_before\train\del"            # 原图路径
for root,dirs,files in os.walk(result_path):
    if files:
        for item in files:
            out_path = root.replace(result_path,dst)
            if not os.path.exists(out_path):
                os.makedirs(out_path)
            try:
                shutil.move(os.path.join(dst,item.replace(".jpg",".jpg")),os.path.join(out_path,item.replace(".jpg",".jpg")))
            except:
                pass
        # shutil.move(os.path.join(dst,item.replace(".jpg",".bmp")),os.path.join(out_path,item.replace(".jpg",".bmp"))) 原图bmp

### 所有缺陷小图按照文件夹分类映射到整图中

In [2]:
img_path = r"\\Ds418\nas3\A-ZK-OQC\soft_data\OQC-0308\result\tmp\1-3_1"
jf = r"\\Ds418\nas3\A-ZK-OQC\soft_data\OQC-0308\result\tmp\batch_0308.json"
crop_size = 120     # 原图上小图裁剪最小尺寸
limit_w = 240       # 小图限制大小
max_w = 2048        # 合并图最高宽度
gen_full_img(img_path,jf,crop_size,limit_w,max_w)

生成缺陷小图参数字典。。。:   0%|          | 0/10 [00:00<?, ?it/s]


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

### 已生成好拼接图拼成大图

In [2]:
img_path = r"\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1"
max_w = 2048
max_size = 480
      
all_imgs = get_no_json_long_img(img_path,max_w,max_size)
long_im = gen_long_img(all_imgs,max_w=max_w)

h,w = long_im.shape[:2]
pad_im = np.ones((h,20,3),dtype=np.uint8)*255
new_im = np.hstack((pad_im,long_im))
cv2.imencode(".jpg",new_im)[1].tofile(os.path.join(img_path,"{}.jpg".format(os.path.basename(img_path))))

\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1\data\1
\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1\data\2
\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1\data\3
\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1\data\4
\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1\data\5
\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1\data\6
\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1\data\7
\\Ds418\NAS\share\LiuPY\OQC\camera_nums_1\train\1_1 & 3_1\data\8


In [5]:
os.remove(r"\\Ds418\nas3\A-PAD\ann_data\20230315\0314\Thumbs.db")

FileNotFoundError: [WinError 2] 系统找不到指定的文件。: '\\\\Ds418\\nas3\\A-PAD\\ann_data\\20230315\\0314\\Thumbs.db'

In [21]:
os.remove(r'\\Ds418\NAS3\A-PAD-new\ann_data\20230412\Thumbs.db')

FileNotFoundError: [WinError 2] 系统找不到指定的文件。: '\\\\Ds418\\NAS3\\A-PAD-new\\ann_data\\20230412\\Thumbs.db'